In [1]:
!wget https://s3.amazonaws.com/fast-ai-nlp/yelp_review_polarity_csv.tgz
!tar zxvf yelp_review_polarity_csv.tgz

--2019-12-09 04:55:06--  https://s3.amazonaws.com/fast-ai-nlp/yelp_review_polarity_csv.tgz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.37.150
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.37.150|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 166373201 (159M) [application/x-tar]
Saving to: ‘yelp_review_polarity_csv.tgz’

yelp_review_polarit 100%[===================>] 158.67M  63.0MB/s    in 2.5s    

2019-12-09 04:55:14 (63.0 MB/s) - ‘yelp_review_polarity_csv.tgz’ saved [166373201/166373201]

yelp_review_polarity_csv/
yelp_review_polarity_csv/train.csv
yelp_review_polarity_csv/readme.txt
yelp_review_polarity_csv/test.csv


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!cp -r drive/My\ Drive/cache_dir cache_dir 

In [4]:
!pip install -q simpletransformers

     |████████████████████████████████| 102kB 3.9MB/s 
     |████████████████████████████████| 645kB 10.9MB/s 
     |████████████████████████████████| 368kB 53.6MB/s 
     |████████████████████████████████| 194kB 76.4MB/s 
     |████████████████████████████████| 1.0MB 52.2MB/s 
     |████████████████████████████████| 860kB 48.3MB/s 


In [5]:
import pandas as pd
prefix = 'yelp_review_polarity_csv/'

train_df = pd.read_csv(prefix + 'train.csv', header=None)
train_df.head()

eval_df = pd.read_csv(prefix + 'test.csv', header=None)
eval_df.head()

train_df[0] = (train_df[0] == 2).astype(int)
eval_df[0] = (eval_df[0] == 2).astype(int)

train_df = pd.DataFrame({
    'text': train_df[1].replace(r'\n', ' ', regex=True),
    'label':train_df[0]
})

print(train_df.head())

eval_df = pd.DataFrame({
    'text': eval_df[1].replace(r'\n', ' ', regex=True),
    'label':eval_df[0]
})

print(eval_df.head())

                                                text  label
0  Unfortunately, the frustration of being Dr. Go...      0
1  Been going to Dr. Goldberg for over 10 years. ...      1
2  I don't know what Dr. Goldberg was like before...      0
3  I'm writing this review to give you a heads up...      0
4  All the food is great here. But the best thing...      1
                                                text  label
0  Contrary to other reviews, I have zero complai...      1
1  Last summer I had an appointment to get new ti...      0
2  Friendly staff, same starbucks fair you get an...      1
3  The food is good. Unfortunately the service is...      0
4  Even when we didn't have a car Filene's Baseme...      1


In [45]:
!git clone https://www.github.com/nvidia/apex
%cd apex
!pip install -q --no-cache-dir ./
#!python setup.py install

fatal: destination path 'apex' already exists and is not an empty directory.
/content/apex


In [46]:
%cd ..
from apex import amp
from simpletransformers.classification import ClassificationModel

/content


In [8]:
model = ClassificationModel('bert', 'bert-base-cased')
#model = ClassificationModel('roberta', 'roberta-base)

100%|██████████| 435779157/435779157 [00:17<00:00, 24595227.67B/s]


In [9]:
# Train the model
model.train_model(train_df,args={'train_batch_size': 110,'eval_batch_size': 110})

Features loaded from cache at cache_dir/cached_train_bert_128_2_560000
Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


Running loss: 0.714348

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:91: UserWarning: Seems like `optimizer.step()` has been overridden after learning rate scheduler initialization. Please, make sure to call `optimizer.step()` before `lr_scheduler.step()`. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Running loss: 0.432277Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0
Running loss: 0.139281
Training of bert model complete. Saved to outputs/.


In [0]:
model2 = ClassificationModel('bert', 'outputs/checkpoint-4000')

In [24]:
model2.train_model(train_df,output_dir='outputs_2/',args={'train_batch_size': 110,'eval_batch_size': 110})

Features loaded from cache at cache_dir/cached_train_bert_128_2_560000
Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


Running loss: 0.063018

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:91: UserWarning: Seems like `optimizer.step()` has been overridden after learning rate scheduler initialization. Please, make sure to call `optimizer.step()` before `lr_scheduler.step()`. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Running loss: 0.103669
Training of bert model complete. Saved to outputs_2/.


In [18]:
# Evaluate the model
import sklearn
result, model_outputs, wrong_predictions = model.eval_model(eval_df, acc=sklearn.metrics.accuracy_score)

Features loaded from cache at cache_dir/cached_dev_bert_128_2_38000


In [32]:
result

{'acc': 0.9568947368421052,
 'eval_loss': 0.11206961244430845,
 'fn': 823,
 'fp': 815,
 'mcc': 0.9137895546849503,
 'tn': 18185,
 'tp': 18177}

In [0]:
model2.args['max_seq_length'] = 450

In [43]:
result_2, model_outputs_2, wrong_predictions_2 = model2.eval_model(eval_df,acc=sklearn.metrics.accuracy_score)

Converting to features started.


In [44]:
result_2

{'acc': 0.9752368421052632,
 'eval_loss': 0.06789457843762774,
 'fn': 528,
 'fp': 413,
 'mcc': 0.9504910946812395,
 'tn': 18587,
 'tp': 18472}